In [1]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

In [2]:
def get_categorymembers(category, level=0, max_level=0, verbose=False):
    """
    Return a list of all wikipedia pages from a given category.
    Categories are themselves pages -> The list includes pages from subcategories that
    are members of a higher level category (recursively).
    
    NOTE: duplicates are not removed.
    
    Args:
    ----
        category (str)
        level (int): current level of the category
        max_level (int): maximum level of the recursion for subcategories
        verbose (bool)
        
    Returns:
    -------
        pages (list): list of `page`.
    
    """
    
    pages = []
    members = category.categorymembers
    for page in members.values():
        info_str = "{0:s}: {1:s} (ns: {2:d})".format('*' * (level + 1), page.title, page.ns)
        if page.ns == wikipediaapi.Namespace.MAIN:
            pages.append(page)
            if verbose:
                print("{0:70.70}{1:>27}".format(info_str, 'PAGE ADDED'))
        if page.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            if verbose:
                print("(SUBCATEGORY) ", info_str)
            get_categorymembers(page, level + 1)
    return pages

In [3]:
# cat = wiki_wiki.page("Category:Science fiction novels by writer")
cat = wiki_wiki.page("Category:Novels by Douglas Adams")
pages = get_categorymembers(cat, verbose=True)

*: Dirk Gently's Holistic Detective Agency (ns: 0)                                     PAGE ADDED
*: The Hitchhiker's Guide to the Galaxy (ns: 0)                                        PAGE ADDED
*: The Hitchhiker's Guide to the Galaxy (novel) (ns: 0)                                PAGE ADDED
*: Life, the Universe and Everything (ns: 0)                                           PAGE ADDED
*: The Long Dark Tea-Time of the Soul (ns: 0)                                          PAGE ADDED
*: Mostly Harmless (ns: 0)                                                             PAGE ADDED
*: The Restaurant at the End of the Universe (ns: 0)                                   PAGE ADDED
*: So Long, and Thanks for All the Fish (ns: 0)                                        PAGE ADDED


In [47]:
import re
import nltk
import gensim
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
from gensim import models, corpora
from pprint import pprint

# nltk.download('all')

In [10]:
print(len(set(STOPWORDS) - set(stopwords.words('english'))))
print(len(set(stopwords.words('english')) - set(STOPWORDS)))

211
53


In [11]:
STOPWORDS_ALL = set(STOPWORDS).union(set(stopwords.words('english')))
print(len(STOPWORDS_ALL))

390


In [12]:
def clean_text(text):
    tokens = word_tokenize(text.lower())
    clean_tokens = [t for t in tokens if (
        t not in STOPWORDS_ALL and
        re.match('[a-zA-Z\-]{3,}', t)
#         re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)
    )]
    return clean_tokens

In [13]:
tokenized_data = []
for p in pages:
    tokenized_data.append(clean_text(p.text))

In [14]:
# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 4th document looks like: [(word_id, count), ...]
# (only the first 10 words)
print(corpus[4][:10])

[(15, 2), (16, 4), (17, 1), (18, 1), (21, 1), (22, 1), (34, 1), (45, 1), (46, 1), (50, 1)]


In [23]:
dictionary[0]

'abend'

In [24]:
NUM_TOPICS = 6

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [34]:
text = tokenized_data[1]
bow = dictionary.doc2bow(text)
 
print(lsi_model[bow])
print(lda_model[bow])

[(0, 240.37361948551592), (1, 11.754043740631356), (2, 10.998284135263896), (3, -1.7140595870982118), (4, -1.337397510248843), (5, 0.3180635755088952)]
[(1, 0.9990717)]


In [49]:
print("LDA Model:")

# Print the Keyword in all topics

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

print("=" * 20)
 
print("LSI Model:")
 
pprint(lsi_model.print_topics())
doc_lsi = lsi_model[corpus]
    
print("=" * 20)

LDA Model:
[(0,
  '0.012*"series" + 0.010*"guide" + 0.010*"adams" + 0.008*"radio" + '
  '0.008*"hitchhiker" + 0.008*"arthur" + 0.007*"book" + 0.007*"earth" + '
  '0.006*"universe" + 0.005*"novel"'),
 (1,
  '0.014*"series" + 0.012*"guide" + 0.010*"arthur" + 0.009*"radio" + '
  '0.009*"hitchhiker" + 0.009*"adams" + 0.009*"galaxy" + 0.008*"book" + '
  '0.007*"earth" + 0.005*"zaphod"'),
 (2,
  '0.012*"series" + 0.010*"adams" + 0.009*"book" + 0.007*"hitchhiker" + '
  '0.007*"radio" + 0.007*"guide" + 0.006*"arthur" + 0.006*"universe" + '
  '0.004*"earth" + 0.004*"dirk"'),
 (3,
  '0.011*"arthur" + 0.009*"series" + 0.008*"hitchhiker" + 0.007*"radio" + '
  '0.007*"book" + 0.007*"adams" + 0.006*"guide" + 0.006*"earth" + 0.006*"ford" '
  '+ 0.005*"universe"'),
 (4,
  '0.014*"series" + 0.010*"arthur" + 0.010*"adams" + 0.009*"radio" + '
  '0.008*"guide" + 0.007*"universe" + 0.006*"book" + 0.006*"hitchhiker" + '
  '0.006*"earth" + 0.004*"zaphod"'),
 (5,
  '0.010*"series" + 0.009*"adams" + 0.008*"rad

In [51]:
doc_lda

In [36]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities)
# [(104, 0.87591344), (178, 0.86124849), (31, 0.8604598), (77, 0.84932965), (85, 0.84843522), (135, 0.84421808), (215, 0.84184396), (353, 0.84038532), (254, 0.83498049), (13, 0.82832891)]

# Let's see what's the most similar document
document_id, similarity = similarities[0]
# print(pages[document_id].text[:1000])
 

[(1, 1.0), (5, 1.0), (7, 1.0), (2, 0.680604), (6, 0.40527242), (0, 0.0), (3, 0.0), (4, 0.0)]


In [146]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()
nltk.download('wordnet')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 2:
#             result.append(lemmatize_stemming(token))
            result.append(token)
    return result

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/bolensadrien/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [147]:
res = preprocess(pages[0].text)

In [160]:
re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', 'aad')

<_sre.SRE_Match object; span=(0, 3), match='aad'>